<a href="https://colab.research.google.com/github/Alankrit151299/ML-Major-may/blob/main/ML_MAJOR_MAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://colab.research.google.com/drive/1p1f3DYvDThWxXqPiKOxglTMDra81ZzGt

In [59]:
!pip install vaderSentiment

In [60]:
!pip install contractions

In [61]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [62]:
import sys

In [63]:
import requests
import pandas as pd
import numpy as np
import os
import nltk
import contractions

In [64]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
def con(text):
  expand = contractions.fix(text)
  return expand 

import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text

from nltk.tokenize.toktok import ToktokTokenizer
tokenizer  = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
# lemmatize string
def lemmatize_word(text):
  word_tokens = word_tokenize(text)
  w = [lemmatizer.lemmatize(word, pos='v') for word in word_tokens]
  lemmas = ' '.join(w) 
  return lemmas


In [67]:
pd.set_option('display.max_colwidth',100)
df = pd.read_csv('https://raw.githubusercontent.com/p1kalys/ML---sklearn-projects/main/Tweets.csv')
df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs ...
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ð..."
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦"


In [68]:
df.tweet = df.tweet.apply(con)
df.tweet = df.tweet.apply(remove_sp)
df.tweet = df.tweet.apply(remove_stopwords)

In [69]:
df['compound'] = df['tweet'].apply(lambda x:vs.polarity_scores(x)['compound'])
df

,id,tweet,compound
0,31963,studiolife aislife requires passion dedication willpower find newmaterials,0.4588
1,31964,user white supremacists want everyone see new birds movie heres,-0.1779
2,31965,safe ways heal acne altwaystoheal healthy healing,0.6808
3,31966,hp cursed child book reservations already yes no harrypotter pottermore favorite,0.5423
4,31967,3rd bihday amazing hilarious nephew eli ahmir uncle dave loves misses,0.8519
...,...,...,...
17192,49155,thought factory leftright polarisation trump uselections2016 leadership politics brexit blm gt3,0.0000
17193,49156,feeling like mermaid hairflip neverready formal wedding gown dresses mermaid,0.4588
17194,49157,hillary campaigned today ohioomg amp used words like assetsampliability never clinton say theewo...,0.3612
17195,49158,happy work conference right mindset leads cultureofdevelopment organizations work mindset,0.5719


In [70]:
df.tweet = df.tweet.apply(lemmatize_word)
df.head()

,id,tweet,compound
0,31963,studiolife aislife require passion dedication willpower find newmaterials,0.4588
1,31964,user white supremacists want everyone see new bird movie heres,-0.1779
2,31965,safe ways heal acne altwaystoheal healthy heal,0.6808
3,31966,hp curse child book reservations already yes no harrypotter pottermore favorite,0.5423
4,31967,3rd bihday amaze hilarious nephew eli ahmir uncle dave love miss,0.8519


In [71]:
for i in range(len(df['compound'])):
  if df.compound[i]<=-0.05:
    df.compound[i] = "negative"
  elif df.compound[i]>=0.05:
    df.compound[i] = "positive"
  elif df.compound[i]>-0.05 and df.compound[i]<0.05:
    df.compound[i] = "neutral"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoi

In [72]:
df.head()

,id,tweet,compound
0,31963,studiolife aislife require passion dedication willpower find newmaterials,positive
1,31964,user white supremacists want everyone see new bird movie heres,negative
2,31965,safe ways heal acne altwaystoheal healthy heal,positive
3,31966,hp curse child book reservations already yes no harrypotter pottermore favorite,positive
4,31967,3rd bihday amaze hilarious nephew eli ahmir uncle dave love miss,positive


In [73]:
df['compound'].value_counts()

positive    8925
neutral     4732
negative    3540
Name: compound, dtype: int64

In [74]:
x = df['tweet'].values
y = df['compound'].values

In [75]:
# Split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

In [76]:
print(x_train.shape)
print(x_test.shape)

(12897,)
(4300,)


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
x_train_vect = vect.fit_transform(x_train)
x_test_vect = vect.transform(x_test)

In [78]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train_vect,y_train) # Map input with output

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [79]:
y_pred = model.predict(x_test_vect)

In [80]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_pred,y_test)

0.8220930232558139

In [81]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.62      0.84      0.71       661
     neutral       0.86      0.74      0.79      1360
    positive       0.89      0.87      0.88      2279

    accuracy                           0.82      4300
   macro avg       0.79      0.81      0.79      4300
weighted avg       0.84      0.82      0.82      4300



In [82]:
from sklearn.pipeline import make_pipeline
text_model = make_pipeline(TfidfVectorizer(),SVC())

text_model.fit(x_train,y_train)

y_pred = text_model.predict(x_test)
y_pred

array(['neutral', 'positive', 'negative', ..., 'positive', 'positive',
       'neutral'], dtype=object)

In [83]:
import joblib
joblib.dump(text_model,'Sentiment_Analysis')

['Sentiment_Analysis']